# Problem set 6: Classification

## Description

Explore different algorithms to classify Old Bailey cases as involving "stealing" or "other."

## Imports and setup

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from   sklearn.decomposition import TruncatedSVD
from   sklearn.feature_extraction.text import TfidfVectorizer

# Our input text file
old_bailey_file = os.path.join('..', '..', 'data', 'old_bailey', 'old_bailey.txt')

## Old Bailey records

We'll work with a set of 3,090 short text documents from the Old Bailey archive, just as we did last week.

In [ ]:
# Read cases in as a list of strings
with open(old_bailey_file, 'r') as f:
    bailey = [doc for doc in f.read().split('\n\n')] # split on consecutive newlines
print("Total documents:", len(bailey))

In [ ]:
# Freebie function to assign gold labels to corpus texts
def make_labels(corpus, keyword='steal'):
    '''
    Takes a corpus of documents and a keyword string.
    Assigns labels based on coöccurrence of keyword and ' indicted for ' in first sentence of document.
    Returns an array of class labels (1=member, 0=nonmember).
    '''
    from nltk import word_tokenize, sent_tokenize
    import re
    find_indictment = re.compile(' indicted for ')
    find_keyword = re.compile(keyword)
    labels = []
    for doc in corpus:
        first_sentence = sent_tokenize(doc)[0].lower()
        if find_indictment.search(first_sentence) and find_keyword.search(first_sentence):
            labels.append(1)
        else:
            labels.append(0)
    return np.array(labels)

In [ ]:
# Generate gold labels
y_bailey = make_labels(bailey, keyword='steal')
print("Label vector shape:", y_bailey.shape)
print("Stealing cases:", np.sum(y_bailey))
print("Fraction of cases labeled 'stealing':", round(np.sum(y_bailey)/len(y_bailey),3))

In [ ]:
# Baseline accuracy and F1
#  What if we just guess 1 every time?
baseline_accuracy = np.sum(y_bailey)/len(y_bailey)
baseline_precision = baseline_accuracy
baseline_recall = 1.0
baseline_f1 = 2*baseline_precision*baseline_recall/(baseline_precision+baseline_recall)
print("Baseline accuracy:", round(baseline_accuracy, 3))
print("Baseline F1:", round(baseline_f1, 3))

## 1. Vectorize (5 points)

Using the vectorizer defined below, transform the input documents into a TFIDF-weighted document-term matrix. Store your vectorized output in a varaible named `X_bailey` and print the shape of the resulting matrix.

Note: This is a straight carry-over from the last problem set.

In [ ]:
# Custom preprocessing to remove escaped characters in input
def pre_proc(x):
    '''
    Takes a unicode string.
    Lowercases, strips accents, and removes some escapes.
    Returns a standardized version of the string.
    '''
    import unicodedata
    return unicodedata.normalize('NFKD', x.replace("\'", "'").replace("\ in\ form", " inform").lower().strip())

# Set up vectorizer
vectorizer = TfidfVectorizer(
    encoding='utf-8',
    preprocessor=pre_proc,
    min_df=2, # Note this
    max_df=0.8, # This, too
    binary=False,
    norm='l2',
    use_idf=True # And this
)

In [ ]:
# Your code here

In [ ]:
# Get the dimensions of the doc-term matrix
print("Matrix shape:", X_bailey.shape)

In [ ]:
# Freebie: plot our data
coords_bailey = TruncatedSVD(n_components=2).fit_transform(X_bailey)

plt.subplots(figsize=(12,8))
sns.scatterplot(
    x=coords_bailey[:, 0], 
    y=coords_bailey[:, 1],
    hue=y_bailey,
    alpha=0.2,
    linewidth=0
)
plt.title('Old Bailey Theft vs. Other')
plt.show()

## Freebies: Classification and cross-validation how-to

Below is a sample of how to set up classifiers and perform cross-validation.

In [ ]:
# Cross-validate two simple classifiers on our data
from sklearn.model_selection import cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# Classifers to test
classifiers = {
    'kNN': KNeighborsClassifier(),
    'Logit':LogisticRegression()
}

scores = {} # Store cross-validation results in a dictionary
for classifier in classifiers: 
    scores[classifier] = cross_validate( # perform cross-validation
        classifiers[classifier], # classifier object
        X_bailey, # feature matrix
        y_bailey, # gold labels
        cv=10, #number of folds
        scoring=['accuracy', 'f1', 'f1_macro', 'f1_micro'] # scoring methods
    )

In [ ]:
# Examine the performance of our simple classifiers
# Freebie function to summarize and display classifier scores
def compare_scores(scores_dict):
    '''
    Takes a dictionary of cross_validate scores.
    Returns a color-coded Pandas dataframe that summarizes those scores.
    '''
    import pandas as pd
    df = pd.DataFrame(scores_dict).T.applymap(np.mean).style.background_gradient(cmap='RdYlGn')
    return df

In [ ]:
# Compare cross-validation scores
# Note that colorization of the `time` columns is counterintuitive
compare_scores(scores)

## 2. Add results from three more classifiers (25 points)

* Set up the three classifiers imported below (decision tree, random forest, and multinomial naïve Bayes). Use **default** parameters only (that is, do not set any classifier parameters yourself). (10 points total)
* Cross-validate the three new classifiers, saving the scoring output to the same `scores` dictionary as above. (10 points)
* Use the `compare_scores` function to display the scores of all five classifiers. (5 points)

This all takes less than 30 seconds total to run on my laptop.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

## 3. Compare the unoptimized performance of the five classifiers (10 points)

How would you summarize the performance of the five classifiers prior to any optimization? Consider classification performance relative to one another and to baseline, as well as computation time. 

**Your comparison here**

## 4. Improve two of the classifiers (40 points)

See how much you can improve the performance of any two of the five classifiers. You might try:

* Changing the vectorization parameters. (Hint: it can help to use fewer input features).
* Changing the classifier parameters. 
    * See the `sklearn` [documentation](https://scikit-learn.org/stable/supervised_learning.html#supervised-learning) for your chosen classifiers for a list of available options.
* Trying a different classifier entirely.
    * If you do this, make sure you give careful consideration to the settings of the new classifier, too. **Using default parameters will result in a score of zero.**
    
No matter what you try, you must **document your process**. In other words, don't just change settings in a cell and run it over and over. You should iterate over different options, storing your cross-validation scores for each combination of settings.

Finally, display your results using the `compare_scores` function.

Points breakdown: 20 points total for each classifier, of which 10 are for iterating over a range of parameter values, 5 are for performing cross validation correctly and displaying summary scores, and 5 are for actually improving performance (by any amount). 

In [ ]:
# Your code here

## 5. Examine and discuss classification errors (20 points)

First, rerun your best-performing classifier over the full corpus and save the resulting labels as `y_best`. (10 points)

To do this, first set up a classifier with your optimized paramters; then use the classifier's `fit` method to train the model; then use the fitted classifer's `predict` method to compute the labels. In schematic form, this looks like:

```
y_labels = Classifier(options).fit(X, y).predict(X)
```

(Note that training and testing over the same data is poor practice, since it encourages overfitting. We avoided that problem by using cross validation above. We're doing it here only for ease of examination.)

Then use the `pull_errors` function below to read through some cases that were incorrectly classified by your best-perfoming classifer. Write a paragraph that summarizes any patterns you can identify in the mis-classified cases. Can you explain what may be confusing the classifier? Does your analysis suggest any avenues for improved performance? (10 points)

In [ ]:
# Your code here

In [ ]:
# Function to examine some classification errors
def pull_errors(labels, gold_labels=y_bailey, corpus=bailey, n=3):
    '''
    Takes:
        an array of computed labels
        an array of gold (correct) labels
        a list of corpus documents
        an int of cases to display
    '''
    import pandas as pd
    labeled_cases = pd.DataFrame(
        {
            'gold':gold_labels, 
            'computed':labels,
            'text':corpus
        }   
    )
    errors = labeled_cases.loc[labeled_cases.gold != labeled_cases.computed]
    with pd.option_context('display.max_colwidth', None):
        display(errors.sample(n))

In [ ]:
# Pull errors
pull_errors(y_best, y_bailey, bailey, n=3)

**Your discussion here**